In [1]:
from braindecode.util import set_random_seeds, np_to_var, var_to_np
import matplotlib.pyplot as plt
from moabb.datasets import BNCI2014001, Cho2017, PhysionetMI
from moabb.paradigms import MotorImagery
import numpy as np
from numpy.random import RandomState
import pickle
import time
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

import mne

import start_kits.util.shallow_net
from start_kits.util.utilfunc import get_balanced_batches
from start_kits.util.preproc import plot_confusion_matrix

from braindecode.models import ShallowFBCSPNet, EEGNetv1, EEGNetv4

cuda = torch.cuda.is_available()
print('gpu: ', cuda)
device = 'cuda' if cuda else 'cpu'

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
rng = RandomState(seed)

/home/ducanh/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


gpu:  True


In [2]:
import numpy as np
x_val = np.load('/home/ducanh/hain/hungp/NeurIPS_BEETL/numpy_data_binary/X_val.npy')

# original shape: (num_trial, num_electrons, num_samples) with each trial is 1 label
input = x_val[:2, :, :]
input = input.astype('float32')
print(input.shape)

# add batch dims
input = np.expand_dims(input, axis=0)

# convert type to torch tensor
input = torch.from_numpy(input)
# reshape input size
input = input.permute(1, 2, 3, 0)

print(input.shape)

(2, 32, 256)
torch.Size([2, 32, 256, 1])


In [ ]:
in_chans = 32

model = EEGNetv4(
    in_chans = in_chans,
    n_classes = 3,
    input_window_samples=256,
)

if cuda:
    model.cuda()
checkpoint = torch.load('cnn_model_mi.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

average_acc, average_loss = [], []
setname = 'testset'
dataset = test_set

i_trials_in_batch = get_balanced_batches(len(dataset.X), rng, shuffle=False,
                                         batch_size=batch_size)
outputs=None
for i_trials in i_trials_in_batch:
    # Have to add empty fourth dimension to X
    batch_X = dataset.X[i_trials][:, :, :, None]
    batch_y = dataset.y[i_trials]
    net_in = np_to_var(batch_X)
    if cuda:
        net_in = net_in.cuda()
    toutputs = model(net_in)
    if outputs is None:
        temp = toutputs.cpu()
        outputs = temp.detach().numpy()
    else:
        temp = toutputs.cpu()
        outputs = np.concatenate((outputs,temp.detach().numpy()))

net_target = np_to_var(dataset.y)
loss = F.nll_loss(torch.from_numpy(outputs), net_target)
print("{:6s} Loss: {:.5f}".format(setname, float(var_to_np(loss))))
predicted_labels = np.argmax((outputs), axis=1)
accuracy = np.mean(dataset.y  == predicted_labels)

print("{:6s} Accuracy: {:.1f}%".format(setname, accuracy * 100))
plot_confusion_matrix(dataset.y, predicted_labels, 
                      classes=['LH','Other'], normalize=True,
                      title='Validation confusion matrix')
plt.show()

In [4]:
in_chans = 32

model = EEGNetv4(
    in_chans = in_chans,
    n_classes = 3,
    input_window_samples=256,
)

'''
TODO: Load model pretrained here
'''
checkpoint = torch.load('cnn_model_mi.pth')
model.load_state_dict(checkpoint['model_state_dict'])

if cuda:
    input = input.cuda()
    model = model.cuda()
model.eval()

RuntimeError: Error(s) in loading state_dict for EEGNetv4:
	size mismatch for conv_spatial.weight: copying a param with shape torch.Size([16, 1, 22, 1]) from checkpoint, the shape in current model is torch.Size([16, 1, 32, 1]).
	size mismatch for conv_classifier.weight: copying a param with shape torch.Size([3, 16, 1, 23]) from checkpoint, the shape in current model is torch.Size([3, 16, 1, 8]).

In [5]:
out = model(input)
out.shape

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same